In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("./data/sudoku.csv")

In [4]:
puzzle_str=df["puzzle"].loc(0)[0]
solution_str=df["solution"].loc(0)[0]
np_solution=np.array(list(solution_str)).reshape(9,9)
print(np_solution)


[['6' '7' '9' '5' '1' '8' '2' '4' '3']
 ['5' '4' '3' '7' '2' '9' '6' '1' '8']
 ['8' '2' '1' '6' '3' '4' '9' '5' '7']
 ['7' '9' '4' '3' '5' '2' '1' '8' '6']
 ['3' '5' '8' '4' '6' '1' '7' '2' '9']
 ['2' '1' '6' '8' '9' '7' '5' '3' '4']
 ['4' '8' '5' '2' '7' '6' '3' '9' '1']
 ['9' '6' '2' '1' '8' '3' '4' '7' '5']
 ['1' '3' '7' '9' '4' '5' '8' '6' '2']]


In [64]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
np_cells

array([['0', '7', '0', '0', '0', '0', '0', '4', '3'],
       ['0', '4', '0', '0', '0', '9', '6', '1', '0'],
       ['8', '0', '0', '6', '3', '4', '9', '0', '0'],
       ['0', '9', '4', '0', '5', '2', '0', '0', '0'],
       ['3', '5', '8', '4', '6', '0', '0', '2', '0'],
       ['0', '0', '0', '8', '0', '0', '5', '3', '0'],
       ['0', '8', '0', '0', '7', '0', '0', '9', '1'],
       ['9', '0', '2', '1', '0', '0', '0', '0', '5'],
       ['0', '0', '7', '0', '4', '0', '8', '0', '2']], dtype='<U1')

In [65]:
# %load sudoku.py

def map2orginal(row, col, block_row, block_col):
    return (block_row*3 +row, block_col*3 + col)

def getBlockIndex(row, col):
    r= row//3
    c = col//3
    return (r,c)

# def getBlock(row, col, np_cells):
#     r, c = getBlockIndex(row,col)
#     return np_cells[r*3:r*3+3, c*3:c*3+3]

def getBlock(block_row, block_col, np_cells):
    return np_cells[block_row*3:block_row*3+3, block_col*3:block_col*3+3]

def getBlockList(block_row, block_col, np_cells):
    return getBlock(block_row, block_col, np_cells).reshape(9)

def getRow(row, np_cells):
    return np_cells[row]

def getCol(col, np_cells):
    return np_cells[:, col]

def calculate(row_index, col_index, np_cells):
    row_index=row_index
    col_index=col_index
    row=getRow(row_index, np_cells)
    col=getCol(col_index, np_cells)
    block_row, block_col = getBlockIndex(row, col)
    block=getBlockList(block_row, block_col, np_cells)
    possibilities={i+1 for i in range(9)}
    for i in row:
        if i in possibilities:
            possibilities.remove(i)
    for j in col:
        if j in possibilities:
            possibilities.remove(j)
    for k in block:
        if k in possibilities:
            possibilities.remove(k)
    return (row_index, col_index, possibilities)

def calculate_cells(np_cells):
    changed=True
    results=[]
    iterations=0
    print(np_cells)
    while changed:
        iterations+=1
        changed=False
        results=[]
        for i in range(9):
            for j in range(9):
                if np_cells[i,j] == 0 :
                    results.append(calculate(i,j,np_cells))


        for x,y,s in results:
            if len(s)==1:
                np_cells[x,y]=next(iter(s))
                changed=True
        
        print(results)
        print("iterations:", iterations)
        print("change:", changed)

    print(np_cells)


In [66]:
#single_solve(np_cells)
b=getBlock(0,0, np_cells)
# getBlockArray(0,0, np_cells)
getBlockIndex(3,3)

(1, 1)

In [96]:
class Block:
    def __init__(self, r, c, np_cells) -> None:
        self.r=r
        self.c=c
        self.block=getBlock(r,c,np_cells)
        self.block_row, self.block_col=getBlockIndex(r,c)
        self.np_cells=np_cells
        self.missing_set={i for i in "123456789"}
        self.missings={}
        self.unknowns=[]
        for i in range(3):
            for j in range(3):
                if self.block[i,j]=='0':
                    self.unknowns.append((i,j))
                else:
                    self.missing_set.remove(self.block[i,j])
        print("unknown:", self.unknowns)
        print(self.missing_set)
        print(self.missings)

        for m in self.missing_set:
            for u in self.unknowns:
                if self.couldfit(m, u):
                    print("add {} to {}".format(u, m))
                    self.addpos(m, u)
                else:
                    print("could not add {} to {}".format(u, m))
        
        print("unknown:", self.unknowns)
        print(self.missing_set)
        print(self.missings)

    def getMissings(self, missing_set, unknowns):
        pass

    def couldfit(self, m, u):
        m_r=u[0]+self.block_row*3
        m_c=u[1]+self.block_col*3
        row=getRow(m_r, self.np_cells)
        #print("row:", row, " m_r:", m_r)
        col=getCol(m_c, self.np_cells)
        #print("col:",col, " m_c:", m_c)
        result =True
        if m in row:
            print("found {} in {}".format(m, row))
            result=False
        if m in col:
            print("found {} in {}".format(m, col))
            result=False
        if m in row or m in col:
            result=False
        return result

    def addpos(self, m, u):
        if m in self.missings.keys():
            poss=self.missings[m]
            poss.append(u)
        else:
            poss=[u]
            self.missings[m]=poss

    def removepos(self,removes):
        deleted=[]
        for r in removes:
            for m in self.missings.keys():
                if r in self.missings[m]:
                    self.missings[m].remove(r)
                    if len(self.missings[m])==0:
                        deleted.append(m)
                    
        for d in deleted:
            self.missings.pop(d)


    def calculate(self):
        removes=[]
        changed=False
        repeat=True
        while repeat:
            repeat=False
            for m in self.missings.keys():
                if len(self.missings[m]) == 1:
                    px, py=self.missings[m][0]
                    self.block[px, py]=m
                    removes.append((px,py))
                    print(self.block)
                    changed=True
                    repeat=False
            self.removepos(removes)
        
        return changed

b=Block(1,1,np_cells)

print(np_cells)
print(b.missing_set)
print(b.missings)
b.calculate()
print(b.missings)
print(np_cells)



    

unknown: [(0, 0), (1, 2), (2, 1), (2, 2)]
{'9', '7', '3', '1'}
{}
found 9 in ['0' '0' '8' '6' '3' '7' '0' '9' '0']
could not add (0, 0) to 9
found 9 in ['0' '4' '0' '0' '0' '9' '6' '1' '0']
could not add (1, 2) to 9
found 9 in ['8' '0' '0' '6' '3' '4' '9' '0' '0']
found 9 in ['7' '4' '0' '9' '5' '0' '8' '0' '0']
could not add (2, 1) to 9
found 9 in ['8' '0' '0' '6' '3' '4' '9' '0' '0']
could not add (2, 2) to 9
found 7 in ['0' '7' '0' '0' '0' '0' '0' '4' '3']
found 7 in ['0' '0' '8' '6' '3' '7' '0' '9' '0']
could not add (0, 0) to 7
found 7 in ['0' '0' '0' '4' '8' '0' '0' '2' '7']
could not add (1, 2) to 7
found 7 in ['7' '4' '0' '9' '5' '0' '8' '0' '0']
could not add (2, 1) to 7
found 7 in ['0' '0' '0' '4' '8' '0' '0' '2' '7']
could not add (2, 2) to 7
found 3 in ['0' '7' '0' '0' '0' '0' '0' '4' '3']
found 3 in ['0' '0' '8' '6' '3' '7' '0' '9' '0']
could not add (0, 0) to 3
add (1, 2) to 3
found 3 in ['8' '0' '0' '6' '3' '4' '9' '0' '0']
could not add (2, 1) to 3
found 3 in ['8' '0' '

In [71]:
def calculate_blocks(np_cells):
    changed=False
    for block_row in range(3):
        for block_col in range(3):
            print("solving {},{}".format(block_row, block_col) )
            b=Block(block_row, block_col, np_cells)
            if b.calculate():
                changed=True
    return changed

def single_solve(np_cells):
    changed=True
    while changed:
        changed=False
        calculate_cells(np_cells)
        if calculate_blocks(np_cells):
            changed=True
    print(np_cells)

In [72]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
single_solve(np_cells)

[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
[]
iterations: 1
change: False
[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
solving 0,0
[['0' '7' '9']
 ['0' '4' '0']
 ['8' '0' '0']]
[['0' '7' '9']
 ['0' '4' '3']
 ['8' '0' '0']]
solving 0,1
[['0' '0' '0']
 ['7' '0' '9']
 ['6' '3' '4']]
solving 0,2
solving 1,0
[['6' '9' '4']
 ['3' '5' '8']
 ['0' '0' '0']]
[['6' '9' '4']
 ['3' '5' '8']
 ['

In [98]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
calculate_cells(np_cells)
b00=Block(0,1,np_cells)
b00.calculate()
print(b.missings)
print(np_cells)

# b01=Block(0,1,np_cells)
# b01.calculate()
# print(b.missings)
# print(np_cells)

[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
[]
iterations: 1
change: False
[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
unknown: [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1)]
{'7', '2', '5', '8', '1'}
{}
found 7 in ['0' '7' '0' '0' '0' '0' '0' '4' '3']
could not add (0, 0) to 7
found 7 in ['0' '7' '0' '0' '0' '0' '0' '4' '3']
found 7 in ['7' '4' '0' '9' '5' '0' '8' '0' '0']
could not add

In [60]:
print(np_cells)
c=getBlock(0,1,np_cells)
c[1,0]='x'
print(np_cells)

[['0' '7' '9' '0' '0' '0' '0' '4' '3']
 ['x' '4' '3' '7' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
[['0' '7' '9' '0' '0' '0' '0' '4' '3']
 ['x' '4' '3' 'x' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
